In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


## Veri Yükleme

In [2]:
df=pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [3]:
df.head().T

,0,1,2,3,4
Time,0.000000,0.000000,1.000000,1.000000,2.000000
V1,-1.359807,1.191857,-1.358354,-0.966272,-1.158233
V2,-0.072781,0.266151,-1.340163,-0.185226,0.877737
V3,2.536347,0.166480,1.773209,1.792993,1.548718
V4,1.378155,0.448154,0.379780,-0.863291,0.403034
V5,-0.338321,0.060018,-0.503198,-0.010309,-0.407193
V6,0.462388,-0.082361,1.800499,1.247203,0.095921
V7,0.239599,-0.078803,0.791461,0.237609,0.592941
V8,0.098698,0.085102,0.247676,0.377436,-0.270533
V9,0.363787,-0.255425,-1.514654,-1.387024,0.817739


## Aykırı Gözlem Analizi

In [4]:
from sklearn.neighbors import LocalOutlierFactor

In [5]:
clf=LocalOutlierFactor(n_neighbors=20, contamination=0.1)

In [6]:
clf

LocalOutlierFactor(contamination=0.1)

In [7]:
clf.fit_predict(df)

array([-1, -1, -1, ..., -1, -1, -1])

In [8]:
df_scores=clf.negative_outlier_factor_

In [9]:
df_scores

array([-1.50991882, -1.35540632, -1.98923712, ..., -1.82634219,
       -1.19589913, -1.50428432])

In [10]:
df_scores[0:20]

array([-1.50991882, -1.35540632, -1.98923712, -1.56601448, -1.83683298,
       -1.30225123, -1.24643823, -1.52264534, -1.68598793, -1.14147396,
       -1.10901312, -1.10820985, -1.44025425, -1.16135197, -1.55450046,
       -1.09489206, -1.07422898, -1.07917131, -1.5340813 , -1.05155589])

In [11]:
np.sort(df_scores)[0:20]

array([-11.80059647, -10.04141425,  -8.6519314 ,  -8.15898588,
        -8.08041487,  -8.07982721,  -6.47023354,  -6.3968558 ,
        -5.85788419,  -5.66483917,  -5.32790327,  -5.29839417,
        -5.08738445,  -4.99879587,  -4.57507771,  -4.53578506,
        -4.50252384,  -4.47814126,  -4.43935263,  -4.42456587])

In [12]:
esik_deger=np.sort(df_scores)[6] # Eşik değer olarak kırılmanın olduğu noktaya bakılmıştır.
# 6 indeksli değer eşik değer olarak seçilmiştir.
esik_deger

-6.47023354043277

In [13]:
aykiri_tf=df_scores>esik_deger 
# Burada df_scores büyüktür eşik değer aldığımız için aykiri_tf içine anlamlı değerler geliyor.

aykiri_tf.shape

(284807,)

In [14]:
aykiri_tf

array([ True,  True,  True, ...,  True,  True,  True])

In [15]:
yeni_df=df[df_scores>esik_deger]  # Burada çıkan değerler eşik değerin üstünde kalan değerler,  
                                  # yani bizim için anlamlı olan verilerdir
yeni_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [16]:
df[~(df_scores>esik_deger)] 
# bunlarda eşik değerin altında kalan değerlerdir. Yani bizim için anlamsız verilerdir.

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1632,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.366110,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
6380,7587.0,1.025341,-0.708149,0.941277,-0.066131,-1.051823,-0.037449,-0.802394,-0.036824,2.185846,...,0.065926,0.173485,-0.207172,-0.399911,0.031019,1.491184,-0.109930,0.026895,149.70,0
6386,7589.0,-2.073411,-0.363452,2.227696,0.689432,-0.186702,-0.155875,-0.524638,0.783487,1.531872,...,0.220138,0.232848,-0.062539,-0.038796,0.409944,-0.355953,-0.078900,-0.151653,105.28,0
151296,95286.0,-34.549296,-60.464618,-21.340854,16.875344,-19.229075,6.335259,24.422716,-4.964566,0.188912,...,11.502580,-9.499423,-16.513186,0.744341,-7.081325,-2.604551,-3.550963,3.250802,18910.00,0
154234,100924.0,-23.984747,16.697832,-22.209875,9.584969,-16.230439,2.596333,-33.239328,-21.560039,-10.842526,...,-12.615023,5.774087,2.750221,0.513411,-1.608804,-0.459624,-4.626127,-0.334561,1.00,1
274771,166198.0,-35.548539,-31.850484,-48.325589,15.304184,-113.743307,73.301626,120.589494,-27.347360,-3.872425,...,-21.620120,5.712303,-1.581098,4.584549,4.554683,3.415636,31.612198,-15.430084,25691.16,0
284249,172273.0,-9.030538,-11.112584,-16.233798,3.592021,-40.427726,23.917837,44.054461,-7.277778,-4.210637,...,-0.269048,0.988144,7.040028,0.347693,2.520869,2.342495,3.478175,-2.713136,10199.44,0


In [17]:
df[df_scores==esik_deger]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1632,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.36611,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0


### 2) Aykırı Değerlere Baskı Değer Atama

In [18]:
# Burada eşik değeri bulduktan sonra baskılama yöntemi ile eşik değerin altında kalan değerleri
# Baskılayacağız.

In [19]:
baski_deger=df[df_scores==esik_deger]

In [20]:
aykirilar=df[~aykiri_tf]

In [21]:
aykirilar

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1632,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.366110,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
6380,7587.0,1.025341,-0.708149,0.941277,-0.066131,-1.051823,-0.037449,-0.802394,-0.036824,2.185846,...,0.065926,0.173485,-0.207172,-0.399911,0.031019,1.491184,-0.109930,0.026895,149.70,0
6386,7589.0,-2.073411,-0.363452,2.227696,0.689432,-0.186702,-0.155875,-0.524638,0.783487,1.531872,...,0.220138,0.232848,-0.062539,-0.038796,0.409944,-0.355953,-0.078900,-0.151653,105.28,0
151296,95286.0,-34.549296,-60.464618,-21.340854,16.875344,-19.229075,6.335259,24.422716,-4.964566,0.188912,...,11.502580,-9.499423,-16.513186,0.744341,-7.081325,-2.604551,-3.550963,3.250802,18910.00,0
154234,100924.0,-23.984747,16.697832,-22.209875,9.584969,-16.230439,2.596333,-33.239328,-21.560039,-10.842526,...,-12.615023,5.774087,2.750221,0.513411,-1.608804,-0.459624,-4.626127,-0.334561,1.00,1
274771,166198.0,-35.548539,-31.850484,-48.325589,15.304184,-113.743307,73.301626,120.589494,-27.347360,-3.872425,...,-21.620120,5.712303,-1.581098,4.584549,4.554683,3.415636,31.612198,-15.430084,25691.16,0
284249,172273.0,-9.030538,-11.112584,-16.233798,3.592021,-40.427726,23.917837,44.054461,-7.277778,-4.210637,...,-0.269048,0.988144,7.040028,0.347693,2.520869,2.342495,3.478175,-2.713136,10199.44,0


In [22]:
type(aykirilar) # aykırıları bulduk, bunlar şu anda data Frame tipinde

pandas.core.frame.DataFrame

In [23]:
aykirilar.to_records(index=False) # burada aykiriları np.dizi yapacağız

rec.array([(  1264., -11.14070636,  -9.6127262 , -12.38954523,  6.01334629,  -32.092129  ,  2.13930688e+01,  34.30317686,  -7.52078427,  -1.92573155,  -2.63662207,  3.70217663,  -1.84305632,  2.42712334, -1.65283566,  3.63504152,  4.08780179,  -1.654892  , -0.88840646,  0.75227637, -11.74868903,  -4.70997725,  1.36610997,  -2.92588754,  0.8435508 ,  0.74626744,  0.80138692,  3.85204564,   4.15793357, 7.712430e+03, 0),
           (  7587.,   1.02534096,  -0.70814886,   0.94127679, -0.06613094,   -1.05182316, -3.74493513e-02,  -0.80239352,  -0.03682449,   2.1858457 ,  -0.67756358, -0.0824551 ,  -2.53838427,  2.29250576,  1.09679451,  1.00540699,  1.13240285,  -0.14651713,  0.73767207, -0.228783  ,   0.26674239,   0.06592625,  0.17348486,  -0.20717236, -0.39991126,  0.03101909,  1.49118394, -0.10992954,   0.02689547, 1.497000e+02, 0),
           (  7589.,  -2.07341113,  -0.36345223,   2.22769641,  0.68943213,   -0.18670162, -1.55874907e-01,  -0.52463759,   0.78348735,   1.53187184,  -1.55

In [24]:
res=aykirilar.to_records(index=False) # buarada aykirilar np dizisi oldu

In [25]:
res[:]

rec.array([(  1264., -11.14070636,  -9.6127262 , -12.38954523,  6.01334629,  -32.092129  ,  2.13930688e+01,  34.30317686,  -7.52078427,  -1.92573155,  -2.63662207,  3.70217663,  -1.84305632,  2.42712334, -1.65283566,  3.63504152,  4.08780179,  -1.654892  , -0.88840646,  0.75227637, -11.74868903,  -4.70997725,  1.36610997,  -2.92588754,  0.8435508 ,  0.74626744,  0.80138692,  3.85204564,   4.15793357, 7.712430e+03, 0),
           (  7587.,   1.02534096,  -0.70814886,   0.94127679, -0.06613094,   -1.05182316, -3.74493513e-02,  -0.80239352,  -0.03682449,   2.1858457 ,  -0.67756358, -0.0824551 ,  -2.53838427,  2.29250576,  1.09679451,  1.00540699,  1.13240285,  -0.14651713,  0.73767207, -0.228783  ,   0.26674239,   0.06592625,  0.17348486,  -0.20717236, -0.39991126,  0.03101909,  1.49118394, -0.10992954,   0.02689547, 1.497000e+02, 0),
           (  7589.,  -2.07341113,  -0.36345223,   2.22769641,  0.68943213,   -0.18670162, -1.55874907e-01,  -0.52463759,   0.78348735,   1.53187184,  -1.55

In [26]:
res[:]=baski_deger.to_records(index=False)

In [27]:
res   # bu değişkenin içine baskılamak için oluşturduğumuz verileri hazırladık. 
      # Şimdi bunu gidip outlier olan datalarımızn olduğu yere atacağız. 
      # Oradaki outliar olan değerler bu değerler ile değişecek

rec.array([(1264., -11.14070636, -9.6127262, -12.38954523, 6.01334629, -32.092129, 21.39306876, 34.30317686, -7.52078427, -1.92573155, -2.63662207, 3.70217663, -1.84305632, 2.42712334, -1.65283566, 3.63504152, 4.08780179, -1.654892, -0.88840646, 0.75227637, -11.74868903, -4.70997725, 1.36610997, -2.92588754, 0.8435508, 0.74626744, 0.80138692, 3.85204564, 4.15793357, 7712.43, 0),
           (1264., -11.14070636, -9.6127262, -12.38954523, 6.01334629, -32.092129, 21.39306876, 34.30317686, -7.52078427, -1.92573155, -2.63662207, 3.70217663, -1.84305632, 2.42712334, -1.65283566, 3.63504152, 4.08780179, -1.654892, -0.88840646, 0.75227637, -11.74868903, -4.70997725, 1.36610997, -2.92588754, 0.8435508, 0.74626744, 0.80138692, 3.85204564, 4.15793357, 7712.43, 0),
           (1264., -11.14070636, -9.6127262, -12.38954523, 6.01334629, -32.092129, 21.39306876, 34.30317686, -7.52078427, -1.92573155, -2.63662207, 3.70217663, -1.84305632, 2.42712334, -1.65283566, 3.63504152, 4.08780179, -1.654892, -0.

In [28]:
df[~aykiri_tf]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1632,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.366110,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
6380,7587.0,1.025341,-0.708149,0.941277,-0.066131,-1.051823,-0.037449,-0.802394,-0.036824,2.185846,...,0.065926,0.173485,-0.207172,-0.399911,0.031019,1.491184,-0.109930,0.026895,149.70,0
6386,7589.0,-2.073411,-0.363452,2.227696,0.689432,-0.186702,-0.155875,-0.524638,0.783487,1.531872,...,0.220138,0.232848,-0.062539,-0.038796,0.409944,-0.355953,-0.078900,-0.151653,105.28,0
151296,95286.0,-34.549296,-60.464618,-21.340854,16.875344,-19.229075,6.335259,24.422716,-4.964566,0.188912,...,11.502580,-9.499423,-16.513186,0.744341,-7.081325,-2.604551,-3.550963,3.250802,18910.00,0
154234,100924.0,-23.984747,16.697832,-22.209875,9.584969,-16.230439,2.596333,-33.239328,-21.560039,-10.842526,...,-12.615023,5.774087,2.750221,0.513411,-1.608804,-0.459624,-4.626127,-0.334561,1.00,1
274771,166198.0,-35.548539,-31.850484,-48.325589,15.304184,-113.743307,73.301626,120.589494,-27.347360,-3.872425,...,-21.620120,5.712303,-1.581098,4.584549,4.554683,3.415636,31.612198,-15.430084,25691.16,0
284249,172273.0,-9.030538,-11.112584,-16.233798,3.592021,-40.427726,23.917837,44.054461,-7.277778,-4.210637,...,-0.269048,0.988144,7.040028,0.347693,2.520869,2.342495,3.478175,-2.713136,10199.44,0


In [29]:
df[~aykiri_tf] = pd.DataFrame(res, index=df[~aykiri_tf].index)

In [30]:
df[~aykiri_tf]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
1632,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.36611,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
6380,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.36611,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
6386,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.36611,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
151296,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.36611,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
154234,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.36611,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
274771,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.36611,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
284249,1264.0,-11.140706,-9.612726,-12.389545,6.013346,-32.092129,21.393069,34.303177,-7.520784,-1.925732,...,-4.709977,1.36611,-2.925888,0.843551,0.746267,0.801387,3.852046,4.157934,7712.43,0
